In [1]:
import csv
import pandas as pd
import numpy as np
import math
import scipy
from sklearn.linear_model import LinearRegression

In [2]:
def dateToIndex(date):
    datearr = date.split("/")
    month = int(datearr[0])
    year = int(datearr[2])
    return (year-2010)*12+month-1

In [3]:
appVolDat = pd.read_csv('filingVolumes.csv')
prediction = appVolDat.iloc[1405]
appVolDat = appVolDat.iloc[1320:1404]
appVolDat = appVolDat.reset_index(drop=True)
maximum = max(appVolDat.volume)
appVolDat.volume = appVolDat.volume/maximum
appVolDat.volume = np.log(appVolDat.volume)
print(appVolDat)

        month    volume
0    1/1/2010 -0.974123
1    2/1/2010 -0.916152
2    3/1/2010 -0.657338
3    4/1/2010 -0.761886
4    5/1/2010 -0.835769
5    6/1/2010 -0.680101
6    7/1/2010 -0.817939
7    8/1/2010 -0.797985
8    9/1/2010 -0.730612
9   10/1/2010 -0.775803
10  11/1/2010 -0.777923
11  12/1/2010 -0.612443
12   1/1/2011 -0.857095
13   2/1/2011 -0.862841
14   3/1/2011 -0.599843
15   4/1/2011 -0.777950
16   5/1/2011 -0.757948
17   6/1/2011 -0.645195
18   7/1/2011 -0.842958
19   8/1/2011 -0.702106
20   9/1/2011 -0.555220
21  10/1/2011 -0.785842
22  11/1/2011 -0.755972
23  12/1/2011 -0.581095
24   1/1/2012 -0.824396
25   2/1/2012 -0.758873
26   3/1/2012 -0.587616
27   4/1/2012 -0.711771
28   5/1/2012 -0.670383
29   6/1/2012 -0.606377
..        ...       ...
54   7/1/2014 -0.646871
55   8/1/2014 -0.683886
56   9/1/2014 -0.555031
57  10/1/2014 -0.604789
58  11/1/2014 -0.773635
59  12/1/2014 -0.518915
60   1/1/2015 -0.786185
61   2/1/2015 -0.802325
62   3/1/2015 -0.576436
63   4/1/2015 -0

In [4]:
for i in range(1,13):
    appVolDat['volShift'+str(i)] = appVolDat.volume.shift(-1*i)

In [5]:
appVolDat

,month,volume,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
0,1/1/2010,-0.974123,-0.916152,-0.657338,-0.761886,-0.835769,-0.680101,-0.817939,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095
1,2/1/2010,-0.916152,-0.657338,-0.761886,-0.835769,-0.680101,-0.817939,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841
2,3/1/2010,-0.657338,-0.761886,-0.835769,-0.680101,-0.817939,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843
3,4/1/2010,-0.761886,-0.835769,-0.680101,-0.817939,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843,-0.777950
4,5/1/2010,-0.835769,-0.680101,-0.817939,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843,-0.777950,-0.757948
5,6/1/2010,-0.680101,-0.817939,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843,-0.777950,-0.757948,-0.645195
6,7/1/2010,-0.817939,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843,-0.777950,-0.757948,-0.645195,-0.842958
7,8/1/2010,-0.797985,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843,-0.777950,-0.757948,-0.645195,-0.842958,-0.702106
8,9/1/2010,-0.730612,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843,-0.777950,-0.757948,-0.645195,-0.842958,-0.702106,-0.555220
9,10/1/2010,-0.775803,-0.777923,-0.612443,-0.857095,-0.862841,-0.599843,-0.777950,-0.757948,-0.645195,-0.842958,-0.702106,-0.555220,-0.785842


In [38]:
start = 0
end = 70
log_model = LinearRegression().fit( 
    
    np.c_[appVolDat.volShift1[start:end], 
          appVolDat.volShift2[start:end], 
          appVolDat.volShift3[start:end], 
          appVolDat.volShift4[start:end], 
          appVolDat.volShift5[start:end], 
          appVolDat.volShift6[start:end], 
          appVolDat.volShift7[start:end], 
          appVolDat.volShift8[start:end], 
          appVolDat.volShift9[start:end], 
          appVolDat.volShift10[start:end],
          appVolDat.volShift11[start:end],
          appVolDat.volShift12[start:end]],   
    appVolDat.volume[start:end])

In [39]:
log_model.coef_

array([ 0.0418971 ,  0.07813302,  0.19704589,  0.02119685,  0.08034048,
        0.11015288,  0.03868669, -0.14001442,  0.07128886, -0.25772322,
       -0.17286829,  0.32834983])

In [40]:
log_model.predict(
    
    np.c_[appVolDat.volShift1[71], 
          appVolDat.volShift2[71], 
          appVolDat.volShift3[71], 
          appVolDat.volShift4[71], 
          appVolDat.volShift5[71], 
          appVolDat.volShift6[71], 
          appVolDat.volShift7[71], 
          appVolDat.volShift8[71], 
          appVolDat.volShift9[71], 
          appVolDat.volShift10[71],
          appVolDat.volShift11[71],
          appVolDat.volShift12[71]]

)

array([-0.91873609])